In [1]:
import fitz  # PyMuPDF
import pdfplumber
import os
import pandas as pd
import markdownify
import base64
from openai import OpenAI
import json

client = OpenAI()

def extract_pdf_elements(pdf_path):
    """
    PDF 파일에서 텍스트, 이미지, 표를 추출하여 별도로 저장하는 함수

    Args:
        pdf_path (str): 처리할 PDF 파일의 경로

    Returns:
        dict: 추출된 텍스트, 이미지 정보, 표 정보가 담긴 딕셔너리
    """
    
    output_dir = os.path.basename(pdf_path).split(".")[0]
    image_dir = os.path.join(output_dir, "images")
    table_dir = os.path.join(output_dir, "tables")
    
    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(table_dir, exist_ok=True)
    
    all_text = []
    image_info = []
    table_info = []

    doc = fitz.open(pdf_path)
    
    with pdfplumber.open(pdf_path) as plumber_pdf:
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            plumber_page = plumber_pdf.pages[page_num]
            
            # 텍스트 추출
            all_text.append(page.get_text("html"))
            
            
            # 이미지 추출
            images = page.get_images(full=True)
            for img_index, img in enumerate(images):
                xref = img[0]
                base_image = doc.extract_image(xref)
                image_bytes = base_image["image"]
                image_ext = base_image["ext"]
                
                image_filename = f"p{page_num + 1}_img{img_index}.{image_ext}"
                image_path = os.path.join(image_dir, image_filename)
                
                with open(image_path, "wb") as img_file:
                    img_file.write(image_bytes)
                    
                    image_info.append({
                        "path":image_path,
                        "page":page_num+1,
                        "description":""
                    })
                    
            # 테이블 추출
            tables = plumber_page.extract_tables()
            for table_index, table in enumerate(tables):
                df = pd.DataFrame(table[1:], columns=table[0])
                
                table_filename = f"p{page_num + 1}_tbl{table_index}.csv"
                table_path = os.path.join(table_dir, table_filename)
                
                df.to_csv(table_path, index=False, encoding="utf-8-sig")
                
                table_info.append({
                    "path":table_path,
                    "page":page_num+1,
                    "description":"",
                    "dataframe":df
                })
                
    doc.close()
    
    full_text = "\n".join(all_text)
    full_markdown = markdownify.markdownify(full_text, heading_style="ATX")
    
    text_filename = os.path.join(output_dir, "full_markdown.txt")
    with open(text_filename, "w", encoding="utf-8") as f:
        f.write(full_markdown)
        
    return {
        "output_dir":output_dir,
        "text_file":text_filename,
        "images":image_info,
        "tables":table_info,
        "text_per_page": full_markdown
    }
    
def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
    
def generate_image_description(image_path):
    base64_image = encode_image_to_base64(image_path)
    image_ext = os.path.splitext(image_path)[1].lstrip('.')
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "이 이미지는 무엇에 관한 것인가요? RAG 시스템에서 검색될 것을 가정하고, 이미지의 핵심 내용을 간결하게 한글로 설명해주세요."},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/{image_ext};base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        max_tokens=200
    )
    return response.choices[0].message.content

def generate_table_summary(table_csv_str):
    if len(table_csv_str) > 4000:
        table_csv_str = table_csv_str[:4000]
        
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": "당신은 데이터 분석가입니다. 주어진 표 데이터를 보고, 표의 핵심 내용을 간결하게 한글 문장으로 요약하는 역할을 맡았습니다."
            },
            {
                "role": "user",
                "content": f"다음은 표 데이터입니다. 이 표가 어떤 정보를 담고 있으며, 주요 특징이 무엇인지 한두 문장으로 요약해주세요.\n\n--- 데이터 시작 ---\n{table_csv_str}\n--- 데이터 끝 ---"
            }
        ],
        max_tokens=300
    )
    return response.choices[0].message.content

In [2]:
pdf_path = "datasets/여비산정기준표.pdf"
extracted_data = extract_pdf_elements(pdf_path)

for i, image_item in enumerate(extracted_data["images"]):
    description = generate_image_description(image_item['path'])
    image_item["description"] = description
    print(description)
    
for i, table_item in enumerate(extracted_data["tables"]):
    description = generate_table_summary(table_item['path'])
    table_item["description"] = description
    print(description)
    

output_dir = extracted_data['output_dir']
json_data_path = os.path.join(output_dir, 'enriched_data.json')
# DataFrame 객체는 JSON으로 바로 저장되지 않으므로 제외
for tbl in extracted_data['tables']:
    del tbl['dataframe']
    
with open(json_data_path, 'w', encoding='utf-8') as f:
    json.dump(extracted_data, f, ensure_ascii=False, indent=4)

죄송합니다. 이 이미지는 회색 배경이라서 구체적인 내용을 알 수 없습니다. 
이 표는 여비 산정에 대한 기준을 담고 있으며, 출장 및 여행 시 필요한 비용 산정의 근거와 기준을 명확히 제시하는 정보를 포함하고 있습니다.
주어진 데이터는 '여비산정기준표'라는 제목을 가진 표로, 여비 산정을 위한 기준이나 규정을 담고 있는 것으로 보입니다. 구체적인 항목이나 내용은 주어지지 않았지만, 여행 경비나 출장을 위한 비용 산정에 관련된 정보를 제공하는 것 같습니다.
표 데이터는 여행 경비를 산정하는 기준을 담고 있는 것으로 보입니다. 주요 특징은 출장이나 여행 시 필요한 경비를 효율적으로 계산할 수 있도록 가이드라인을 제공하는 것입니다.
제공된 표는 여비 산정을 위한 기준을 담고 있으며, 출장 시 교통비, 숙박비 등의 경비를 산정하는 표준을 제시하고 있습니다. 주요 특징은 각 항목별로 세부 산정 기준이 명시되어 있다는 점입니다.
표 데이터는 여비 산정에 관한 기준을 제시하고 있는 것으로 보입니다. 주로 출장 등에 필요한 여비를 계산하는 데 필요한 기준과 요율이 포함되어 있을 가능성이 높습니다.


In [3]:
content_map = {
    i + 1: [page_text] for i, page_text in enumerate(extracted_data["text_per_page"])
}

with open(json_data_path, 'r', encoding='utf-8') as f:
    enriched_data = json.load(f)
    
for image in enriched_data["images"]:
    page_num = image["page"]
    description = image["description"]
    
    formatted_desc = f"\n[이미지 파일: {os.path.basename(image['path'])}]\n이미지 설명: {description}\n"
    if page_num in content_map:
        content_map[page_num].append(formatted_desc)
        
for table in enriched_data["tables"]:
    page_num = table["page"]
    summary = table["description"]
    
    formatted_summary = f"\n[표 파일: {os.path.basename(table['path'])}]\n표 요약: {summary}\n"
    if page_num in content_map:
        content_map[page_num].append(formatted_summary)

In [4]:
final_pages_content = []
for page_num in sorted(content_map.keys()):
    page_content = '\n'.join(content_map[page_num])
    final_pages_content.append(page_content)

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 마크다운에 적합한 스플리터
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=100,
    # 마크다운 구조를 우선적으로 고려하여 분리
    separators=["\n## ", "\n### ", "\n#### ", "\n\n", "\n", " ", ""],
    length_function=len
)

all_chunks = []
for i, page_text in enumerate(final_pages_content):
    page_num = i+1
    chunks = text_splitter.create_documents(
        texts=[page_text],
        metadatas=[{"source_page":page_num}]
    )
    all_chunks.extend(chunks)

In [9]:
import torch
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

embedding_model = HuggingFaceEmbeddings(
    model_name="dragonkue/BGE-m3-ko",
    model_kwargs={'device': device},
    encode_kwargs={'normalize_embeddings': True},
)

vectorscore = Chroma.from_documents(
    documents=all_chunks,
    embedding=embedding_model,
    persist_directory="./chroma_db"
)

ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.

In [7]:
import sys
import requests
from PyQt6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout, QPushButton, QLabel, 
    QFileDialog, QLineEdit, QTextEdit, QFrame
)
from PyQt6.QtCore import Qt

class DragDropLabel(QLabel):
    """드래그 앤 드롭을 지원하는 커스텀 라벨 위젯"""
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.setText("여기에 PDF 파일을 드래그 앤 드롭 하거나\n아래 버튼을 클릭하세요.")
        self.setStyleSheet("""
            QLabel {
                border: 2px dashed #aaa;
                padding: 20px;
                font-size: 14px;
                background-color: #f9f9f9;
            }
        """)

    def dragEnterEvent(self, event):
        if event.mimeData().hasUrls():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        for url in event.mimeData().urls():
            file_path = url.toLocalFile()
            if file_path.lower().endswith('.pdf'):
                print(f"드롭된 파일: {file_path}")
                self.parent().upload_file(file_path)
                return
        print("PDF 파일이 아닙니다.")

class RAGClientApp(QWidget):
    def __init__(self):
        super().__init__()
        # Flask 서버 주소
        self.upload_url = "http://127.0.0.1:5000/upload"
        self.ask_url = "http://127.0.0.1:5000/ask"
        self.init_ui()

    def init_ui(self):
        """애플리케이션의 전체 UI를 설정합니다."""
        main_layout = QVBoxLayout()

        # --- 1. 파일 업로드 섹션 ---
        upload_group_label = QLabel("1. PDF 파일 업로드")
        upload_group_label.setStyleSheet("font-size: 16px; font-weight: bold; margin-top: 10px;")
        main_layout.addWidget(upload_group_label)
        
        self.drop_label = DragDropLabel(self)
        main_layout.addWidget(self.drop_label)

        self.btn_select = QPushButton("컴퓨터에서 PDF 파일 선택")
        self.btn_select.clicked.connect(self.open_file_dialog)
        main_layout.addWidget(self.btn_select)
        
        self.status_label = QLabel("상태: 대기 중")
        self.status_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        main_layout.addWidget(self.status_label)
        
        # --- 구분선 ---
        separator = QFrame()
        separator.setFrameShape(QFrame.Shape.HLine)
        separator.setFrameShadow(QFrame.Shadow.Sunken)
        main_layout.addWidget(separator)

        # --- 2. 질문/답변 섹션 ---
        qa_group_label = QLabel("2. 질문하기")
        qa_group_label.setStyleSheet("font-size: 16px; font-weight: bold; margin-top: 10px;")
        main_layout.addWidget(qa_group_label)

        # 질문 입력 레이아웃
        question_layout = QHBoxLayout()
        self.question_input = QLineEdit()
        self.question_input.setPlaceholderText("업로드된 PDF에 대해 질문을 입력하세요...")
        question_layout.addWidget(self.question_input)

        self.btn_ask = QPushButton("질문하기")
        self.btn_ask.clicked.connect(self.ask_question)
        question_layout.addWidget(self.btn_ask)
        main_layout.addLayout(question_layout)

        # 답변 출력창
        answer_label = QLabel("답변:")
        main_layout.addWidget(answer_label)
        
        self.answer_output = QTextEdit()
        self.answer_output.setReadOnly(True) # 답변은 수정할 수 없도록 설정
        self.answer_output.setStyleSheet("background-color: #f0f0f0;")
        main_layout.addWidget(self.answer_output)
        
        # 초기에는 질문 섹션을 비활성화
        self.set_qa_section_enabled(False)

        self.setLayout(main_layout)
        self.setWindowTitle("PDF 기반 RAG 질문 답변 시스템")
        self.setGeometry(300, 300, 500, 600)

    def set_qa_section_enabled(self, enabled):
        """질문/답변 섹션의 활성화 상태를 설정합니다."""
        self.question_input.setEnabled(enabled)
        self.btn_ask.setEnabled(enabled)
        self.answer_output.setEnabled(enabled)
        if not enabled:
            self.answer_output.setPlaceholderText("PDF 파일을 먼저 업로드해주세요.")
        else:
            self.answer_output.setPlaceholderText("")


    def open_file_dialog(self):
        file_path, _ = QFileDialog.getOpenFileName(self, "PDF 파일 선택", "", "PDF Files (*.pdf)")
        if file_path:
            self.upload_file(file_path)

    def upload_file(self, file_path):
        filename = file_path.split('/')[-1]
        self.status_label.setText(f"상태: '{filename}' 업로드 및 처리 중... (파일 크기에 따라 시간이 걸릴 수 있습니다)")
        self.set_qa_section_enabled(False) # 처리 중 다시 비활성화
        QApplication.processEvents() # UI가 멈추지 않도록 이벤트 처리

        try:
            with open(file_path, 'rb') as f:
                files = {'file': (filename, f, 'application/pdf')}
                # RAG 처리는 오래 걸릴 수 있으므로 timeout을 넉넉하게 설정
                response = requests.post(self.upload_url, files=files, timeout=300)
            
            response.raise_for_status()
            
            response_data = response.json()
            self.status_label.setText(f"상태: {response_data.get('message', '업로드 및 처리 완료!')}")
            self.set_qa_section_enabled(True) # 처리가 끝나면 질문 섹션 활성화

        except requests.exceptions.RequestException as e:
            self.status_label.setText(f"상태: 업로드 실패 - {e}")
        except Exception as e:
            self.status_label.setText(f"상태: 오류 발생 - {e}")

    def ask_question(self):
        question = self.question_input.text().strip()
        if not question:
            self.answer_output.setText("질문을 입력해주세요.")
            return

        self.answer_output.setText("답변을 생성 중입니다...")
        QApplication.processEvents()

        try:
            payload = {"question": question}
            response = requests.post(self.ask_url, json=payload, timeout=60)
            response.raise_for_status()

            response_data = response.json()
            self.answer_output.setText(response_data.get("answer", "답변을 가져올 수 없습니다."))

        except requests.exceptions.RequestException as e:
            self.answer_output.setText(f"오류: 서버와 통신할 수 없습니다.\n{e}")
        except Exception as e:
            self.answer_output.setText(f"알 수 없는 오류가 발생했습니다.\n{e}")


if __name__ == '__main__':
    try:
        import requests
    except ImportError:
        print("requests 라이브러리가 필요합니다. 'pip install -r requirements.txt' 명령어로 설치해주세요.")
        sys.exit(1)
        
    app = QApplication(sys.argv)
    ex = RAGClientApp()
    ex.show()
    sys.exit(app.exec())



2025-06-14 22:03:20.273 python[64743:7789898] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'


SystemExit: 0

/opt/miniconda3/envs/rag/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
